# 02 — Multi-Person (Group Photo) Demo

This notebook demonstrates the pipeline's **automatic multi-person handling**:

- Detects **all** persons in a group image
- Runs segmentation per-person and unions masks
- Produces a combined subject mask and point cloud

You should see `summary.json` report `num_people >= 2` and `mode = all`.

In [ ]:
# --- Standard imports ---
import os
from pathlib import Path
import json
import numpy as np

# --- Visualization ---
import matplotlib.pyplot as plt

# Make plots bigger by default (no fixed colors here)
plt.rcParams["figure.figsize"] = (10, 6)

# --- Repo root detection (works when notebook is in ./notebooks) ---
REPO_ROOT = Path.cwd()
if (REPO_ROOT / "src").exists() and (REPO_ROOT / "configs").exists():
    pass
elif (REPO_ROOT.parent / "src").exists() and (REPO_ROOT.parent / "configs").exists():
    REPO_ROOT = REPO_ROOT.parent
else:
    raise RuntimeError("Could not locate repo root. Open this notebook from inside the repo folder.")

print("Repo root:", REPO_ROOT)

# Ensure src is importable
import sys
SRC_DIR = REPO_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# Imports from the project
from human3d.utils.config import load_config
from human3d.pipeline import Human3DPipeline

## 1) Choose a group photo

Pick an image with 2+ people. The pipeline will automatically switch to multi-person mode.

In [ ]:
INPUT_PATH = REPO_ROOT / "data" / "group.jpg"  # change this
assert INPUT_PATH.exists(), f"Input image not found: {INPUT_PATH}"
print("Using input:", INPUT_PATH)

## 2) Run the pipeline

In [ ]:
CFG_PATH = REPO_ROOT / "configs" / "pipeline.yaml"
cfg = load_config(str(CFG_PATH))

pipe = Human3DPipeline(cfg)
out_dir = Path(pipe.run(str(INPUT_PATH)))

print("Outputs saved to:", out_dir)

## 3) Confirm multi-person mode from summary.json

In [ ]:
summary_path = out_dir / "summary.json"
summary = json.loads(Path(summary_path).read_text(encoding="utf-8"))

print(json.dumps(summary, indent=2)[:2000], "\n...")

print("\nKey fields:")
print("mode:", summary.get("mode"))
print("num_people:", summary.get("pose", {}).get("num_people"))

## 4) Visualize outputs

In [ ]:
from PIL import Image

def show_img(p: Path, title: str):
    if not p.exists():
        print(f"[MISS] {p.name} not found")
        return
    img = Image.open(p)
    plt.figure()
    plt.imshow(img)
    plt.axis("off")
    plt.title(title)
    plt.show()

show_img(out_dir / "pose_overlay.png", "Pose overlay (all persons)")
show_img(out_dir / "seg_mask.png", "Union segmentation mask (all persons)")
show_img(out_dir / "seg_overlay.png", "Segmentation overlay")
show_img(out_dir / "depth.png", "Depth visualization (MiDaS)")

## 5) Point cloud preview (optional)

In [ ]:
pc_path = out_dir / "pointcloud.ply"
print("PLY:", pc_path)

try:
    import open3d as o3d
    pcd = o3d.io.read_point_cloud(str(pc_path))
    print(pcd)
    o3d.visualization.draw_geometries([pcd])
except Exception as e:
    print("Open3D preview failed:", repr(e))
    print("Open the PLY file in CloudCompare or MeshLab instead.")